In [20]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import umap
import hdbscan
from matplotlib.lines import Line2D
import urllib.request
from goatools.obo_parser import GODag
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def calculate_tfidf_and_similarity(df, view_name):
    print(f"\n--- Elaborazione Vista: {view_name} ---")
    n_genes = df.shape[0]
    
    doc_freq = df.sum(axis=0) 
    
    idf = np.log(n_genes / doc_freq + 1e-10)
    
    print("   Calcolo similarità (può richiedere qualche secondo)...")
    
    weighted_matrix = df.values * np.sqrt(idf.values)
    
    weighted_intersection = np.dot(weighted_matrix, weighted_matrix.T)
    
    gene_sums = (df * idf).sum(axis=1).values
    
    weighted_union = gene_sums[:, None] + gene_sums[None, :] - weighted_intersection
    
    with np.errstate(divide='ignore', invalid='ignore'):
        similarity = weighted_intersection / weighted_union
        similarity[np.isnan(similarity)] = 0.0
    
    np.fill_diagonal(similarity, 1.0)
    
    sim_df = pd.DataFrame(similarity, index=df.index, columns=df.index)
    
    print(f"   Matrice completata: {sim_df.shape}")
    return sim_df

input_files = {
    "BP": "filtered_BP.csv",
    "CC": "filtered_CC.csv",
    "MF": "filtered_MF.csv",
    "HPO": "filtered_HPO.csv" 
}

similarity_results = {}

for key, filename in input_files.items():
    if os.path.exists(filename):
        # Carica dati
        df = pd.read_csv(filename, index_col=0)
        
        # Calcola
        sim_matrix = calculate_tfidf_and_similarity(df, key)
        
        # Salva
        output_file = f"similarity_{key}.csv"
        sim_matrix.to_csv(output_file)
        print(f"   -> Salvato in: {output_file}")
        
        similarity_results[key] = sim_matrix
    else:
        print(f"ATTENZIONE: File {filename} non trovato. Hai eseguito la Fase 1?")

print("\nFase 2 Completata.")


--- Elaborazione Vista: BP ---
   Calcolo similarità (può richiedere qualche secondo)...
   Matrice completata: (5183, 5183)
   -> Salvato in: similarity_BP.csv

--- Elaborazione Vista: CC ---
   Calcolo similarità (può richiedere qualche secondo)...
   Matrice completata: (5183, 5183)
   -> Salvato in: similarity_CC.csv

--- Elaborazione Vista: MF ---
   Calcolo similarità (può richiedere qualche secondo)...
   Matrice completata: (5183, 5183)
   -> Salvato in: similarity_MF.csv

--- Elaborazione Vista: HPO ---
   Calcolo similarità (può richiedere qualche secondo)...
   Matrice completata: (5183, 5183)
   -> Salvato in: similarity_HPO.csv

Fase 2 Completata.


In [ ]:
def analyze_view(sim_matrix_path, view_name):
    print(f"\n--- Analisi Vista: {view_name} ---")
    
    sim_df = pd.read_csv(sim_matrix_path, index_col=0)
    
    distance_matrix = 1 - sim_df.values
    distance_matrix[distance_matrix < 0] = 0
    
    reducer = umap.UMAP(
        n_neighbors=30,
        min_dist=0.1,
        n_components=2,
        metric='precomputed',
        random_state=42
    )
    
    embedding = reducer.fit_transform(distance_matrix)
    
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=30,
        metric='euclidean',
        cluster_selection_method='eom'
    )
    
    cluster_labels = clusterer.fit_predict(embedding)
    
    n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
    n_noise = list(cluster_labels).count(-1)
    print(f"   -> Trovati {n_clusters} cluster.")
    print(f"   -> Geni scartati come rumore: {n_noise}")

    plt.figure(figsize=(10, 8))
    
    # Definizione colori
    noise_color = (0.8, 0.8, 0.8)
    palette = sns.color_palette('tab20', n_colors=n_clusters)
    cluster_colors = [palette[x] if x >= 0 else noise_color for x in cluster_labels]
    
    plt.scatter(
        embedding[:, 0], 
        embedding[:, 1], 
        c=cluster_colors, 
        s=5, 
        alpha=0.6
    )
    
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='Noise', 
               markerfacecolor=noise_color, markersize=10),
    ]
    plt.legend(handles=legend_elements, loc='upper right')

    plt.title(f'UMAP Projection - {view_name} ({n_clusters} clusters)', fontsize=16)
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')
    
    plt.savefig(f"plot_umap_{view_name}.png", dpi=300)
    plt.close()
    
    results = pd.DataFrame({
        'Gene': sim_df.index,
        'Cluster': cluster_labels,
        'UMAP_1': embedding[:, 0],
        'UMAP_2': embedding[:, 1]
    })
    results.to_csv(f"clusters_{view_name}.csv", index=False)
    
    return n_clusters, n_noise

files = {
    "BP": "similarity_BP.csv",
    "CC": "similarity_CC.csv",
    "MF": "similarity_MF.csv",
    "HPO": "similarity_HPO.csv"
}

summary = []
for name, path in files.items():
    n_clust, n_noise = analyze_view(path, name)
    summary.append({'View': name, 'Clusters': n_clust, 'Noise_Genes': n_noise})

print(pd.DataFrame(summary))

In [22]:
obo_files = {
    "GO": {"url": "https://current.geneontology.org/ontology/go-basic.obo", "path": "go-basic.obo"},
    "HPO": {"url": "https://raw.githubusercontent.com/obophenotype/human-phenotype-ontology/master/hp.obo", "path": "hp.obo"}
}

data_files = {
    "BP": r"C:\Users\nicki\Desktop\magi\Anno 1\Q1\ScientificVisualization\csv_data\gene_go_matrix_propT_rel-is_a-part_of_ont-BP.csv",
    "CC": r"C:\Users\nicki\Desktop\magi\Anno 1\Q1\ScientificVisualization\csv_data\gene_go_matrix_propT_rel-is_a-part_of_ont-CC.csv",
    "MF": r"C:\Users\nicki\Desktop\magi\Anno 1\Q1\ScientificVisualization\csv_data\gene_go_matrix_propT_rel-is_a-part_of_ont-MF.csv",
    "HPO": r"C:\Users\nicki\Desktop\magi\Anno 1\Q1\ScientificVisualization\csv_data\gene_hpo_matrix_binary_withAncestors_namespace_Phenotypic_abnormality.csv"
}

depth_files = {
    "BP": "./csv_data/goterm_depth_propT_rel-is_a-part_of_ont-BP.csv",
    "CC": "./csv_data/goterm_depth_propT_rel-is_a-part_of_ont-CC.csv",
    "MF": "./csv_data/goterm_depth_propT_rel-is_a-part_of_ont-MF.csv"
}

def check_and_download(url, filename):
    if not os.path.exists(filename):
        try:
            urllib.request.urlretrieve(url, filename)
        except Exception as e:
            print(e)

for key, info in obo_files.items():
    check_and_download(info["url"], info["path"])


def load_ontology(obo_path):
    try:
        return GODag(obo_path)
    except Exception as e:
        print(e)
        return None

def normalize_id(term_id):
    term_id = str(term_id)
    if "GO" in term_id or "HP" in term_id:
        return term_id.replace(".", ":").replace("_", ":")
    return term_id

def filter_by_depth(df, depth_file, min_depth=4):
    
    try:
        df_depth = pd.read_csv(depth_file, index_col=0, nrows=1)
        depth_series = pd.to_numeric(df_depth.iloc[0], errors='coerce').dropna()
        valid_terms = set(depth_series[depth_series >= min_depth].index)
        
        cols_to_keep = []
        for col in df.columns:
            if col in valid_terms or normalize_id(col) in valid_terms:
                cols_to_keep.append(col)
                
        return df[cols_to_keep]
    except Exception as e:
        print(e)
        return df

def frequency_filtering(df, min_genes=3, max_pct=0.20):
    counts = df.sum(axis=0)
    limit = df.shape[0] * max_pct
    mask = (counts >= min_genes) & (counts <= limit)
    df_filtered = df.loc[:, mask]
    return df_filtered

def remove_semantic_redundancy(df, dag, threshold=0.7):

    example_col = df.columns[0]
    normalized_ex = normalize_id(example_col)
    if normalized_ex not in dag:
        print(f"Il termine '{example_col}' (norm: '{normalized_ex}') non è stato trovato nel DAG!")


    matrix = (df.values > 0).astype(int).T
    intersect = matrix @ matrix.T
    row_sums = matrix.sum(axis=1)
    union = row_sums[:, None] + row_sums[None, :] - intersect
    
    with np.errstate(divide='ignore', invalid='ignore'):
        sim_matrix = np.triu(intersect / union, k=1)

    pairs = np.where(sim_matrix >= threshold)
    to_drop = set()
    cols = df.columns
    
    match_count = 0
    
    for i, j in zip(*pairs):
        term_a_raw = cols[i]
        term_b_raw = cols[j]
        
        term_a = normalize_id(term_a_raw)
        term_b = normalize_id(term_b_raw)
        
        if term_a in dag and term_b in dag:
            match_count += 1
            parents_b = dag[term_b].get_all_parents()
            parents_a = dag[term_a].get_all_parents()
            
            if term_a in parents_b:
                to_drop.add(term_a_raw)
            elif term_b in parents_a:
                to_drop.add(term_b_raw)
            
    return df.drop(columns=list(to_drop))

def keep_common_active_genes(dfs_dict):
    print("--- Filtro Geni Comuni Attivi ---")
    
    # 1. Trova i geni che hanno almeno un valore != 0 in OGNI vista
    valid_genes_per_view = []
    
    for name, df in dfs_dict.items():
        # Calcola la somma per riga (gene)
        row_sums = df.sum(axis=1)
        # Tieni solo i geni con somma > 0
        active_genes = set(row_sums[row_sums > 0].index)
        valid_genes_per_view.append(active_genes)
        print(f"   Vista {name}: {len(active_genes)} geni attivi su {len(df)}")

    common_genes = set.intersection(*valid_genes_per_view)
    common_genes = sorted(list(common_genes))
    
    print(f"   -> Geni validi comuni rimasti: {len(common_genes)}")
    
    filtered_dict = {}
    for name, df in dfs_dict.items():
        # .loc seleziona solo le righe dei geni comuni
        filtered_dict[name] = df.loc[common_genes]
        
    return filtered_dict

go_dag = load_ontology(obo_files["GO"]["path"])
hpo_dag = load_ontology(obo_files["HPO"]["path"])

processed_dfs = {}

for key, path in data_files.items():
    try:
        df = pd.read_csv(path, index_col=0)
        
        if key in depth_files:
            df = filter_by_depth(df, depth_files[key], min_depth=4)
        df = frequency_filtering(df)
        
        current_dag = hpo_dag if key == "HPO" else go_dag
        
        df = remove_semantic_redundancy(df, current_dag, threshold=0.7)
        
        processed_dfs[key] = df
        
    except FileNotFoundError:
        print(f"File non trovato: {path}")

if processed_dfs:
    final_dfs = keep_common_active_genes(processed_dfs)
    print("\n=== Salvataggio ===")
    for key, df in final_dfs.items():
        out_name = f"filtered_final_{key}.csv"
        df.to_csv(out_name)
        print(f"   -> {key}: {df.shape} salvato in {out_name}")

go-basic.obo: fmt(1.2) rel(2025-10-10) 42,666 Terms
hp.obo: fmt(1.2) rel(hp/2025-11-24) 23,288 Terms
      Coppie simili trovate nel DAG: 1085
      Termini ridondanti (Antenati) rimossi: 753
      Coppie simili trovate nel DAG: 66
      Termini ridondanti (Antenati) rimossi: 36
      Coppie simili trovate nel DAG: 194
      Termini ridondanti (Antenati) rimossi: 122
      Coppie simili trovate nel DAG: 9066
      Termini ridondanti (Antenati) rimossi: 833
--- Filtro Geni Comuni Attivi ---
   Vista BP: 5011 geni attivi su 5183
   Vista CC: 4221 geni attivi su 5183
   Vista MF: 4034 geni attivi su 5183
   Vista HPO: 5162 geni attivi su 5183
   -> Geni validi comuni rimasti: 3317

=== Salvataggio ===
   -> BP: (3317, 4807) salvato in filtered_final_BP.csv
   -> CC: (3317, 461) salvato in filtered_final_CC.csv
   -> MF: (3317, 911) salvato in filtered_final_MF.csv
   -> HPO: (3317, 6038) salvato in filtered_final_HPO.csv


In [24]:
input_files = {
    "BP": "filtered_final_BP.csv",
    "CC": "filtered_final_CC.csv",
    "MF": "filtered_final_MF.csv",
    "HPO": "filtered_final_HPO.csv"
}

def process_phase2(file_path, view_name):
    print(f"\n--- Elaborazione Vista: {view_name} ---")
    
    # 1. Caricamento Dati (Matrice Binaria)
    try:
        df_binary = pd.read_csv(file_path, index_col=0)
        print(f"   Input (Binario): {df_binary.shape}")
    except FileNotFoundError:
        print(f"   ERRORE: File {file_path} non trovato. Salto.")
        return

    tfidf_transformer = TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True)
    
    tfidf_matrix_sparse = tfidf_transformer.fit_transform(df_binary)
    
    df_tfidf = pd.DataFrame(
        tfidf_matrix_sparse.toarray(), 
        index=df_binary.index, 
        columns=df_binary.columns
    )
    print(f"   TF-IDF Calcolato. Range valori: [{df_tfidf.values.min():.4f}, {df_tfidf.values.max():.4f}]")

    # 3. Output per MOFA (Gene x Termini, Pesato)
    mofa_filename = f"mofa_input_{view_name}.csv"
    df_tfidf.to_csv(mofa_filename)

    cosine_sim = cosine_similarity(tfidf_matrix_sparse)
    
    # Ricostruzione DataFrame simmetrico Gene x Gene
    df_sim = pd.DataFrame(
        cosine_sim,
        index=df_binary.index,
        columns=df_binary.index
    )
    
    # Check integrità (Diagonale deve essere 1.0)
    diag_mean = np.diag(df_sim).mean()
    print(f"   Similarità Calcolata: {df_sim.shape}. Media diagonale: {diag_mean:.2f} (atteso 1.0)")

    snf_filename = f"snf_similarity_{view_name}.csv"
    df_sim.to_csv(snf_filename)

for key, filename in input_files.items():
    process_phase2(filename, key)


--- Elaborazione Vista: BP ---
   Input (Binario): (3317, 4807)
   TF-IDF Calcolato. Range valori: [0.0000, 1.0000]
   Similarità Calcolata: (3317, 3317). Media diagonale: 1.00 (atteso 1.0)

--- Elaborazione Vista: CC ---
   Input (Binario): (3317, 461)
   TF-IDF Calcolato. Range valori: [0.0000, 1.0000]
   Similarità Calcolata: (3317, 3317). Media diagonale: 1.00 (atteso 1.0)

--- Elaborazione Vista: MF ---
   Input (Binario): (3317, 911)
   TF-IDF Calcolato. Range valori: [0.0000, 1.0000]
   Similarità Calcolata: (3317, 3317). Media diagonale: 1.00 (atteso 1.0)

--- Elaborazione Vista: HPO ---
   Input (Binario): (3317, 6038)
   TF-IDF Calcolato. Range valori: [0.0000, 1.0000]
   Similarità Calcolata: (3317, 3317). Media diagonale: 1.00 (atteso 1.0)


In [25]:
def analyze_view(sim_matrix_path, view_name):
    print(f"\n--- Analisi Vista: {view_name} ---")
    
    sim_df = pd.read_csv(sim_matrix_path, index_col=0)
    
    distance_matrix = 1 - sim_df.values
    distance_matrix[distance_matrix < 0] = 0
    
    reducer = umap.UMAP(
        n_neighbors=30,
        min_dist=0.1,
        n_components=2,
        metric='precomputed',
        random_state=42
    )
    
    embedding = reducer.fit_transform(distance_matrix)
    
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=30,
        metric='euclidean',
        cluster_selection_method='eom'
    )
    
    cluster_labels = clusterer.fit_predict(embedding)
    
    n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
    n_noise = list(cluster_labels).count(-1)
    print(f"   -> Trovati {n_clusters} cluster.")
    print(f"   -> Geni scartati come rumore: {n_noise}")

    plt.figure(figsize=(10, 8))
    
    # Definizione colori
    noise_color = (0.8, 0.8, 0.8)
    palette = sns.color_palette('tab20', n_colors=n_clusters)
    cluster_colors = [palette[x] if x >= 0 else noise_color for x in cluster_labels]
    
    plt.scatter(
        embedding[:, 0], 
        embedding[:, 1], 
        c=cluster_colors, 
        s=5, 
        alpha=0.6
    )
    
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='Noise', 
               markerfacecolor=noise_color, markersize=10),
    ]
    plt.legend(handles=legend_elements, loc='upper right')

    plt.title(f'UMAP Projection - {view_name} ({n_clusters} clusters)', fontsize=16)
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')
    
    plt.savefig(f"plot_umap_{view_name}.png", dpi=300)
    plt.close()
    
    results = pd.DataFrame({
        'Gene': sim_df.index,
        'Cluster': cluster_labels,
        'UMAP_1': embedding[:, 0],
        'UMAP_2': embedding[:, 1]
    })
    results.to_csv(f"clusters_{view_name}.csv", index=False)
    
    return n_clusters, n_noise

files = {
    "BP": "snf_similarity_BP.csv",
    "CC": "snf_similarity_CC.csv",
    "MF": "snf_similarity_MF.csv",
    "HPO": "snf_similarity_HPO.csv"
}

summary = []
for name, path in files.items():
    n_clust, n_noise = analyze_view(path, name)
    summary.append({'View': name, 'Clusters': n_clust, 'Noise_Genes': n_noise})

print(pd.DataFrame(summary))


--- Analisi Vista: BP ---


C:\Users\nicki\AppData\Roaming\Python\Python312\site-packages\umap\umap_.py:1865: UserWarning: using precomputed metric; inverse_transform will be unavailable
  warn("using precomputed metric; inverse_transform will be unavailable")
C:\Users\nicki\AppData\Roaming\Python\Python312\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
C:\Users\nicki\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\nicki\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


   -> Trovati 23 cluster.
   -> Geni scartati come rumore: 1632

--- Analisi Vista: CC ---


C:\Users\nicki\AppData\Roaming\Python\Python312\site-packages\umap\umap_.py:1865: UserWarning: using precomputed metric; inverse_transform will be unavailable
  warn("using precomputed metric; inverse_transform will be unavailable")
C:\Users\nicki\AppData\Roaming\Python\Python312\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
C:\Users\nicki\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\nicki\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


   -> Trovati 31 cluster.
   -> Geni scartati come rumore: 992

--- Analisi Vista: MF ---


C:\Users\nicki\AppData\Roaming\Python\Python312\site-packages\umap\umap_.py:1865: UserWarning: using precomputed metric; inverse_transform will be unavailable
  warn("using precomputed metric; inverse_transform will be unavailable")
C:\Users\nicki\AppData\Roaming\Python\Python312\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
C:\Users\nicki\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\nicki\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


   -> Trovati 34 cluster.
   -> Geni scartati come rumore: 838

--- Analisi Vista: HPO ---


C:\Users\nicki\AppData\Roaming\Python\Python312\site-packages\umap\umap_.py:1865: UserWarning: using precomputed metric; inverse_transform will be unavailable
  warn("using precomputed metric; inverse_transform will be unavailable")
C:\Users\nicki\AppData\Roaming\Python\Python312\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
C:\Users\nicki\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\nicki\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


   -> Trovati 21 cluster.
   -> Geni scartati come rumore: 959
  View  Clusters  Noise_Genes
0   BP        23         1632
1   CC        31          992
2   MF        34          838
3  HPO        21          959
